In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv("../../../Divar.csv")

C:\Users\Rumi\AppData\Local\Temp\ipykernel_28648\2225313960.py:6: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../../Divar.csv")


In [15]:
pd.set_option('display.max_columns', None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 61 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1000000 non-null  int64  
 1   cat2_slug                   1000000 non-null  object 
 2   cat3_slug                   999999 non-null   object 
 3   city_slug                   999998 non-null   object 
 4   neighborhood_slug           437139 non-null   object 
 5   created_at_month            1000000 non-null  object 
 6   user_type                   288882 non-null   object 
 7   description                 1000000 non-null  object 
 8   title                       999946 non-null   object 
 9   rent_mode                   352994 non-null   object 
 10  rent_value                  351322 non-null   float64
 11  rent_to_single              19 non-null       object 
 12  rent_type                   103961 non-null   object 
 13

In [ ]:
df = df.drop(columns=["user_type" , 
                       "description" ,
                       "title" ,
                       "rent_mode" ,
                       "rent_to_single" ,
                       "rent_type" , 
                       "price_mode", 
                       "credit_mode",
                       "rent_credit_transform",
                       "transformable_price",
                       "transformable_credit",
                       "transformable_credit",
                       "transformable_rent",
                       "transformed_credit",
                       "transformed_rent",
                       "deed_type",
                       "has_business_deed",
                       "total_floors_count",
                       "has_water",
                       "has_electricity",
                       "has_gas",
                       "has_restroom",
                       "building_direction",
                       "floor_material",
                       "property_type",
                       "regular_person_capacity",
                       "extra_person_capacity",
                       "cost_per_extra_person",
                       "rent_price_on_regular_days",
                       "rent_price_on_special_days",
                       "rent_price_at_weekends",
                       "location_radius"
                       ])

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 30 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Unnamed: 0               1000000 non-null  int64  
 1   cat2_slug                1000000 non-null  object 
 2   cat3_slug                999999 non-null   object 
 3   city_slug                999998 non-null   object 
 4   neighborhood_slug        437139 non-null   object 
 5   created_at_month         1000000 non-null  object 
 6   rent_value               351322 non-null   float64
 7   price_value              568346 non-null   float64
 8   credit_value             352095 non-null   float64
 9   land_size                186396 non-null   float64
 10  building_size            980394 non-null   float64
 11  floor                    541748 non-null   object 
 12  rooms_count              845899 non-null   object 
 13  unit_per_floor           302283 non-null   

<div dir="rtl">

پر کردن lat و long با استفاده از داده های شهر و محله و حذف آن ها</br>
ساخت سطون ماه و سال شمسی و حذف سطون ماه میلادی

In [17]:
#Fill lat and long

# Neighborhood-level medians
neigh_geo = (
    df
    .groupby("neighborhood_slug")[["location_latitude", "location_longitude"]]
    .median()
)

# City-level medians
city_geo = (
    df
    .groupby("city_slug")[["location_latitude", "location_longitude"]]
    .median()
)


mask_lat = df["location_latitude"].isna()
mask_lon = df["location_longitude"].isna()

df.loc[mask_lat, "location_latitude"] = (
    df.loc[mask_lat, "neighborhood_slug"]
      .map(neigh_geo["location_latitude"])
)

df.loc[mask_lon, "location_longitude"] = (
    df.loc[mask_lon, "neighborhood_slug"]
      .map(neigh_geo["location_longitude"])
)



mask_lat = df["location_latitude"].isna()
mask_lon = df["location_longitude"].isna()

df.loc[mask_lat, "location_latitude"] = (
    df.loc[mask_lat, "city_slug"]
      .map(city_geo["location_latitude"])
)

df.loc[mask_lon, "location_longitude"] = (
    df.loc[mask_lon, "city_slug"]
      .map(city_geo["location_longitude"])
)



df = df.dropna(subset=["location_latitude", "location_longitude"])



In [18]:
import jdatetime

df["created_at_month"] = df["created_at_month"].astype("datetime64[ns]")

def gregorian_dt_to_jalali_year_month(dt):
    j = jdatetime.date.fromgregorian(
        year=dt.year,
        month=dt.month,
        day=dt.day
    )
    return pd.Series([j.year, j.month])


df[["created_jalali_year", "created_jalali_month"]] = (
    df["created_at_month"]
    .apply(gregorian_dt_to_jalali_year_month)
)


df["created_jalali_year"] = df["created_jalali_year"].astype(int)
df["created_jalali_month"] = df["created_jalali_month"].astype(int)

In [19]:
df = df.drop(
    columns=[
        "city_slug",
        "neighborhood_slug",
        "created_at_month"
        ])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999998 entries, 0 to 999999
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               999998 non-null  int64  
 1   cat2_slug                999998 non-null  object 
 2   cat3_slug                999997 non-null  object 
 3   rent_value               351321 non-null  float64
 4   price_value              568345 non-null  float64
 5   credit_value             352094 non-null  float64
 6   land_size                186395 non-null  float64
 7   building_size            980392 non-null  float64
 8   floor                    541747 non-null  object 
 9   rooms_count              845897 non-null  object 
 10  unit_per_floor           302283 non-null  object 
 11  has_balcony              506410 non-null  object 
 12  has_elevator             541748 non-null  object 
 13  has_warehouse            728153 non-null  object 
 14  has_parki


## پر کردن داده های خالی

In [ ]:
# land_size
# برای ویلایی ها فقط پره پس سایز زمین آپارتمان ها رو با متراژ ساختمان یکی می کنیم
df.loc[df["land_size"].isna(),"land_size"] = df.loc[df["land_size"].isna(),"building_size"]
# حذف آگهی هایی که اندازه زمین ندارند
df = df.dropna(subset=["building_size"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980392 entries, 0 to 999999
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               980392 non-null  int64  
 1   cat2_slug                980392 non-null  object 
 2   cat3_slug                980391 non-null  object 
 3   rent_value               351319 non-null  float64
 4   price_value              567063 non-null  float64
 5   credit_value             352092 non-null  float64
 6   land_size                980392 non-null  float64
 7   building_size            980392 non-null  float64
 8   floor                    541747 non-null  object 
 9   rooms_count              845897 non-null  object 
 10  unit_per_floor           302283 non-null  object 
 11  has_balcony              506410 non-null  object 
 12  has_elevator             541748 non-null  object 
 13  has_warehouse            728153 non-null  object 
 14  has_parki

In [26]:
#floor
#Data is in string so we turn it into int

df['floor'] = df['floor'].astype(str)

#    r'(-?\d+)' means: look for an optional minus sign (-?), followed by digits (\d+)
df['floor'] = df['floor'].str.extract(r'(-?\d+)')[0]

#نال ها را به -2 تبدیل کردیم چون اکثرا زمین های خالی و اجاره روزانه بودن و از جنس بقیه طبقات نیستن
df['floor'] = df['floor'].fillna(-2)

df['floor'] = df['floor'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980392 entries, 0 to 999999
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               980392 non-null  int64  
 1   cat2_slug                980392 non-null  object 
 2   cat3_slug                980391 non-null  object 
 3   rent_value               351319 non-null  float64
 4   price_value              567063 non-null  float64
 5   credit_value             352092 non-null  float64
 6   land_size                980392 non-null  float64
 7   building_size            980392 non-null  float64
 8   floor                    980392 non-null  int64  
 9   rooms_count              845897 non-null  object 
 10  unit_per_floor           302283 non-null  object 
 11  has_balcony              506410 non-null  object 
 12  has_elevator             541748 non-null  object 
 13  has_warehouse            728153 non-null  object 
 14  has_parki

In [29]:
#rooms_count
# They are Persian so we need to turn them to int

room_mapping = {
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'چهار': 4,
    'پنج یا بیشتر': 5,
    'بدون اتاق': 0
}

df['rooms_count'] = df['rooms_count'].replace(room_mapping)

# استفاده از -1 برای زمین و سوییت ها
df['rooms_count'] = pd.to_numeric(df['rooms_count'], errors='coerce').fillna(-1).astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980392 entries, 0 to 999999
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               980392 non-null  int64  
 1   cat2_slug                980392 non-null  object 
 2   cat3_slug                980391 non-null  object 
 3   rent_value               351319 non-null  float64
 4   price_value              567063 non-null  float64
 5   credit_value             352092 non-null  float64
 6   land_size                980392 non-null  float64
 7   building_size            980392 non-null  float64
 8   floor                    980392 non-null  int64  
 9   rooms_count              980392 non-null  int64  
 10  unit_per_floor           302283 non-null  object 
 11  has_balcony              506410 non-null  object 
 12  has_elevator             541748 non-null  object 
 13  has_warehouse            728153 non-null  object 
 14  has_parki

In [33]:
#unit_per_floor
#Drop for now 
df = df.drop(columns=["unit_per_floor"])

KeyError: "['unit_per_floor'] not found in axis"

In [32]:
#پر کدن فیتجر های بولی با فالس
boolean_cols = [
    'has_balcony', 'has_elevator', 'has_warehouse', 'has_parking',
    'is_rebuilt', 'has_security_guard', 'has_barbecue', 
    'has_pool', 'has_jacuzzi', 'has_sauna'
]

df[boolean_cols] = df[boolean_cols].fillna(False).astype(bool)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980392 entries, 0 to 999999
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               980392 non-null  int64  
 1   cat2_slug                980392 non-null  object 
 2   cat3_slug                980391 non-null  object 
 3   rent_value               351319 non-null  float64
 4   price_value              567063 non-null  float64
 5   credit_value             352092 non-null  float64
 6   land_size                980392 non-null  float64
 7   building_size            980392 non-null  float64
 8   floor                    980392 non-null  int64  
 9   rooms_count              980392 non-null  int64  
 10  has_balcony              980392 non-null  bool   
 11  has_elevator             980392 non-null  bool   
 12  has_warehouse            980392 non-null  bool   
 13  has_parking              980392 non-null  bool   
 14  construct

C:\Users\Rumi\AppData\Local\Temp\ipykernel_28648\2041201147.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[boolean_cols] = df[boolean_cols].fillna(False).astype(bool)


In [ ]:
#construction_year

#digits are persian, we need to turn to int


persian_digits = '۰۱۲۳۴۵۶۷۸۹'
english_digits = '0123456789'
translation_table = str.maketrans(persian_digits, english_digits)

df['construction_year'] = (
    df['construction_year']
    .astype(str)
    .str.translate(translation_table)
    .str.extract(r'(\d+)')[0]
)

df['construction_year'] = pd.to_numeric(df['construction_year'])

# 4. Fill 'plot-old' nulls

min_year = df['construction_year'].min()

mask_plot_old = (df['cat3_slug'] == 'plot-old') & (df['construction_year'].isna())
df.loc[mask_plot_old, 'construction_year'] = min_year - 1

# 5. Drop remaining nulls and convert to int
df = df.dropna(subset=['construction_year']).copy()
df['construction_year'] = df['construction_year'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949382 entries, 1 to 999999
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               949382 non-null  int64  
 1   cat2_slug                949382 non-null  object 
 2   cat3_slug                949382 non-null  object 
 3   rent_value               350927 non-null  float64
 4   price_value              566654 non-null  float64
 5   credit_value             351696 non-null  float64
 6   land_size                949382 non-null  float64
 7   building_size            949382 non-null  float64
 8   floor                    949382 non-null  int64  
 9   rooms_count              949382 non-null  int64  
 10  has_balcony              949382 non-null  bool   
 11  has_elevator             949382 non-null  bool   
 12  has_warehouse            949382 non-null  bool   
 13  has_parking              949382 non-null  bool   
 14  construct

In [ ]:
# has_warm_water_provider 
# has_heating_system 
# has_cooling_system
# cat2_slug 
# drop for now

df1 = df.drop(columns=[
    "has_warm_water_provider", 
    "has_heating_system" ,
     "has_cooling_system",
     "cat2_slug" ])

df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949382 entries, 1 to 999999
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            949382 non-null  int64  
 1   cat3_slug             949382 non-null  object 
 2   rent_value            350927 non-null  float64
 3   price_value           566654 non-null  float64
 4   credit_value          351696 non-null  float64
 5   land_size             949382 non-null  float64
 6   building_size         949382 non-null  float64
 7   floor                 949382 non-null  int64  
 8   rooms_count           949382 non-null  int64  
 9   has_balcony           949382 non-null  bool   
 10  has_elevator          949382 non-null  bool   
 11  has_warehouse         949382 non-null  bool   
 12  has_parking           949382 non-null  bool   
 13  construction_year     949382 non-null  int64  
 14  is_rebuilt            949382 non-null  bool   
 15  has_s

In [44]:
bool_cols = df1.select_dtypes(include=['bool']).columns

df1[bool_cols] = df1[bool_cols].astype(int)

df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949382 entries, 1 to 999999
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            949382 non-null  int64  
 1   cat3_slug             949382 non-null  object 
 2   rent_value            350927 non-null  float64
 3   price_value           566654 non-null  float64
 4   credit_value          351696 non-null  float64
 5   land_size             949382 non-null  float64
 6   building_size         949382 non-null  float64
 7   floor                 949382 non-null  int64  
 8   rooms_count           949382 non-null  int64  
 9   has_balcony           949382 non-null  int64  
 10  has_elevator          949382 non-null  int64  
 11  has_warehouse         949382 non-null  int64  
 12  has_parking           949382 non-null  int64  
 13  construction_year     949382 non-null  int64  
 14  is_rebuilt            949382 non-null  int64  
 15  has_s

## جدا سازی اجاره و خرید

In [53]:
# RENT 
df_rent = df1[((df1['rent_value'].notna() | df1['credit_value'].notna()) & df1['price_value'].isna())].copy()

# SELL
df_sell = df1[(df1['price_value'].notna() & (df1['rent_value'].isna()))].copy()

df_sell = df_sell.drop(columns=['rent_value', 'credit_value'])
df_rent = df_rent.drop(columns=['price_value'])

df_rent[['rent_value', 'credit_value']] = df_rent[['rent_value', 'credit_value']].fillna(0)

#delete index
df_sell = df_sell.drop(columns=['Unnamed: 0'], errors='ignore')
df_rent = df_rent.drop(columns=['Unnamed: 0'], errors='ignore')


## حذف داده های پرت

In [62]:
# --- 1. CLEANING df_sell ---
# Filter price_value: 1 milion <= price <= 10 trilion
price_min = 1e6
price_max = 10e12

df_sell = df_sell[
    (df_sell['price_value'] >= price_min) & 
    (df_sell['price_value'] <= price_max)
].copy()


# Filter rent_value <= 10 billion
# Filter credit_value <= 1 trilion
rent_max = 10e9
credit_max = 1e12

df_rent = df_rent[
    (df_rent['rent_value'] <= rent_max) & 
    (df_rent['credit_value'] <= credit_max)
].copy()

# Print results to see how many rows remain
print(f"Remaining Sell listings: {df_sell.shape[0]}")
print(f"Remaining Rent listings: {df_rent.shape[0]}")


Remaining Sell listings: 557335
Remaining Rent listings: 351200


## تبدیل رهن و اجاره به رهن کامل

In [63]:
df_rent['target_credit'] = df_rent['credit_value'] + df_rent['rent_value'] * 30

df_rent = df_rent.drop(columns=["rent_value", "credit_value"])



## Encoding cat3

In [68]:
df_rent.cat3_slug.value_counts()

cat3_slug
apartment-rent                        211395
house-villa-rent                       64410
shop-rent                              45623
office-rent                            21171
industry-agriculture-business-rent      8595
plot-old                                   3
shop-sell                                  2
office-sell                                1
Name: count, dtype: int64

In [69]:
# Define allowed categories for Rent
allowed_rent = ['apartment-rent', 'house-villa-rent', 'shop-rent', 'office-rent', 'industry-agriculture-business-rent']
df_rent = df_rent[df_rent['cat3_slug'].isin(allowed_rent)].copy()

# Define allowed categories for Sell
allowed_sell = ['apartment-sell', 'house-villa-sell', 'plot-old', 'shop-sell', 'industry-agriculture-business-sell', 'office-sell']
df_sell = df_sell[df_sell['cat3_slug'].isin(allowed_sell)].copy()

In [70]:
df_rent.cat3_slug.value_counts()

cat3_slug
apartment-rent                        211395
house-villa-rent                       64410
shop-rent                              45623
office-rent                            21171
industry-agriculture-business-rent      8595
Name: count, dtype: int64

In [71]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoders (using separate ones for safety, though one is fine)
le_rent = LabelEncoder()
le_sell = LabelEncoder()

# Apply to the columns
df_rent['cat3_slug'] = le_rent.fit_transform(df_rent['cat3_slug'])
df_sell['cat3_slug'] = le_sell.fit_transform(df_sell['cat3_slug'])

# Quick check: print the mapping so you know which number is which
print("Rent Mapping:", dict(zip(le_rent.classes_, le_rent.transform(le_rent.classes_))))
print("Sell Mapping:", dict(zip(le_sell.classes_, le_sell.transform(le_sell.classes_))))

Rent Mapping: {'apartment-rent': np.int64(0), 'house-villa-rent': np.int64(1), 'industry-agriculture-business-rent': np.int64(2), 'office-rent': np.int64(3), 'shop-rent': np.int64(4)}
Sell Mapping: {'apartment-sell': np.int64(0), 'house-villa-sell': np.int64(1), 'industry-agriculture-business-sell': np.int64(2), 'office-sell': np.int64(3), 'plot-old': np.int64(4), 'shop-sell': np.int64(5)}


## Train test Split

In [72]:
from sklearn.model_selection import train_test_split

# --- 1. Split SELL Data ---
X_sell = df_sell.drop(columns=['price_value'])
y_sell = df_sell['price_value']

X_sell_train, X_sell_test, y_sell_train, y_sell_test = train_test_split(
    X_sell, y_sell, test_size=0.2, random_state=42
)

# --- 2. Split RENT Data ---
X_rent = df_rent.drop(columns=['target_credit'])
y_rent = df_rent['target_credit']

X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(
    X_rent, y_rent, test_size=0.2, random_state=42
)

print(f"Sell Train Shape: {X_sell_train.shape}")
print(f"Rent Train Shape: {X_rent_train.shape}")

Sell Train Shape: (445868, 20)
Rent Train Shape: (280955, 20)


In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# --- SELL DATA MODELS ---
# Linear Regression
lr_sell = LinearRegression()
lr_sell.fit(X_sell_train, y_sell_train)
lr_sell_pred = lr_sell.predict(X_sell_test)

# Decision Tree
dt_sell = DecisionTreeRegressor(random_state=42)
dt_sell.fit(X_sell_train, y_sell_train)
dt_sell_pred = dt_sell.predict(X_sell_test)

# --- RENT DATA MODELS ---
# Linear Regression
lr_rent = LinearRegression()
lr_rent.fit(X_rent_train, y_rent_train)
lr_rent_pred = lr_rent.predict(X_rent_test)

# Decision Tree
dt_rent = DecisionTreeRegressor(random_state=42)
dt_rent.fit(X_rent_train, y_rent_train)
dt_rent_pred = dt_rent.predict(X_rent_test)

# --- RESULTS ---
print("--- SELL DATA RESULTS ---")
print(f"Linear Regression R2: {r2_score(y_sell_test, lr_sell_pred):.4f}")
print(f"Decision Tree R2:     {r2_score(y_sell_test, dt_sell_pred):.4f}")

print("\n--- RENT DATA RESULTS ---")
print(f"Linear Regression R2: {r2_score(y_rent_test, lr_rent_pred):.4f}")
print(f"Decision Tree R2:     {r2_score(y_rent_test, dt_rent_pred):.4f}")



--- SELL DATA RESULTS ---
Linear Regression R2: 0.0035
Decision Tree R2:     -1.2936

--- RENT DATA RESULTS ---
Linear Regression R2: 0.0101
Decision Tree R2:     -0.3575


## log10 scale

In [81]:
from sklearn.ensemble import RandomForestRegressor

# 1. Transform targets to Log Scale
y_sell_train_log = np.log1p(y_sell_train)
y_sell_test_log = np.log1p(y_sell_test)

y_rent_train_log = np.log1p(y_rent_train)
y_rent_test_log = np.log1p(y_rent_test)

# 2. Use a Random Forest (Limited depth to prevent the negative R2/Overfit)
# We use a smaller forest first just to see if it improves
rf_sell = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
rf_sell.fit(X_sell_train, y_sell_train_log)

# 3. Predict and transform back (or evaluate in log space)
sell_pred_log = rf_sell.predict(X_sell_test)
print(f"New SELL Random Forest R2 (Log Space): {r2_score(y_sell_test_log, sell_pred_log):.4f}")

# Repeat for Rent
rf_rent = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
rf_rent.fit(X_rent_train, y_rent_train_log)
rent_pred_log = rf_rent.predict(X_rent_test)
print(f"New RENT Random Forest R2 (Log Space): {r2_score(y_rent_test_log, rent_pred_log):.4f}")

New SELL Random Forest R2 (Log Space): 0.3958
New RENT Random Forest R2 (Log Space): 0.4943


In [77]:


# --- SELL DATA MODELS ---
# Linear Regression (Log)
lr_sell = LinearRegression()
lr_sell.fit(X_sell_train, y_sell_train_log)
lr_sell_pred_log = lr_sell.predict(X_sell_test)

# Decision Tree (Log)
dt_sell = DecisionTreeRegressor(random_state=42)
dt_sell.fit(X_sell_train, y_sell_train_log)
dt_sell_pred_log = dt_sell.predict(X_sell_test)

# ---  RENT DATA MODELS ---
# Linear Regression (Log)
lr_rent = LinearRegression()
lr_rent.fit(X_rent_train, y_rent_train_log)
lr_rent_pred_log = lr_rent.predict(X_rent_test)

# Decision Tree (Log)
dt_rent = DecisionTreeRegressor(random_state=42)
dt_rent.fit(X_rent_train, y_rent_train_log)
dt_rent_pred_log = dt_rent.predict(X_rent_test)

# --- RESULTS (Calculated in Log Space) ---
print("--- SELL DATA RESULTS (Log Scale) ---")
print(f"Linear Regression R2: {r2_score(y_sell_test_log, lr_sell_pred_log):.4f}")
print(f"Decision Tree R2:     {r2_score(y_sell_test_log, dt_sell_pred_log):.4f}")

print("\n--- RENT DATA RESULTS (Log Scale) ---")
print(f"Linear Regression R2: {r2_score(y_rent_test_log, lr_rent_pred_log):.4f}")
print(f"Decision Tree R2:     {r2_score(y_rent_test_log, dt_rent_pred_log):.4f}")

--- SELL DATA RESULTS (Log Scale) ---
Linear Regression R2: 0.1798
Decision Tree R2:     -0.1100

--- RENT DATA RESULTS (Log Scale) ---
Linear Regression R2: 0.2332
Decision Tree R2:     0.0296


## ایحاد تغییرات مفهومی

In [82]:
df_rent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 351194 entries, 2 to 999999
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   cat3_slug             351194 non-null  int64  
 1   land_size             351194 non-null  float64
 2   building_size         351194 non-null  float64
 3   floor                 351194 non-null  int64  
 4   rooms_count           351194 non-null  int64  
 5   has_balcony           351194 non-null  int64  
 6   has_elevator          351194 non-null  int64  
 7   has_warehouse         351194 non-null  int64  
 8   has_parking           351194 non-null  int64  
 9   construction_year     351194 non-null  int64  
 10  is_rebuilt            351194 non-null  int64  
 11  has_security_guard    351194 non-null  int64  
 12  has_barbecue          351194 non-null  int64  
 13  has_pool              351194 non-null  int64  
 14  has_jacuzzi           351194 non-null  int64  
 15  has_s